# Парсинг гороскопов в архивах сайтов

## 1. Сайт https://sibmama.ru

На этом сайте есть архив гороскопов на каждый день недели для всех занков зодиак за 2022-2016 годы. Попробуем собрать все данные и с привязкой к знаку Зодиака.
Напишем функцию для сбора url по неделям и для получения данных за каждый день

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
from datetime import date
from csv import writer
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [4]:
def get_url(url):
    #url = f'https://sibmama.ru/index.htm?r=406'
    response = requests.get(url, headers = {'user-agent': ua.random})
    soup = BeautifulSoup(response.content, 'lxml')
    data = soup.find_all(class_ = 'pgs')
    for item in data:
        ur = 'https://sibmama.ru/' + item.find('a').attrs['href']
        yield ur

def get_item(url):
    horos = []
    for url in get_url(url):
        #print(url)
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        #data = soup.find('div')

        days = soup.find_all('h3')
        for day in days:
            
            zods = day.find_previous_siblings()
            try:
                for zod in zods:
                    if (zod.name == 'h2') and (zod.text != '') and (day.text == 'Понедельник'):
                        zodiac = zod.text
                        break
            except:
                zodiac = 'Mistake'
            hors = day.find_next_sibling()
            
            try:
                horos.append(zodiac + '*' + hors.text)
            except:
                horos.append('Mistake')
                    
    yield horos

Для каждого года на сайте предусмотрен отдельный адрес, переберем их в цикле и запишем все списки в общий для последующей обработки.

In [5]:
url_list = ['https://sibmama.ru/index.htm?r=406',
            'https://sibmama.ru/index.htm?r=392',
            'https://sibmama.ru/index.htm?r=382',
            'https://sibmama.ru/index.htm?r=368',
            'https://sibmama.ru/index.htm?r=357',
            'https://sibmama.ru/index.htm?r=305',
            'https://sibmama.ru/index.htm?r=283',
            'https://sibmama.ru/index.htm?r=235'
           ]

total = []
for url in url_list:
    print(f'Log: working over {url} ...')
    total.extend(get_item(url))
len(total) 

Log: working over https://sibmama.ru/index.htm?r=406 ...
Log: working over https://sibmama.ru/index.htm?r=392 ...
Log: working over https://sibmama.ru/index.htm?r=382 ...
Log: working over https://sibmama.ru/index.htm?r=368 ...
Log: working over https://sibmama.ru/index.htm?r=357 ...
Log: working over https://sibmama.ru/index.htm?r=305 ...
Log: working over https://sibmama.ru/index.htm?r=283 ...
Log: working over https://sibmama.ru/index.htm?r=235 ...


8

In [36]:
flat_list = [item for sublist in total for item in sublist]
df = pd.DataFrame(flat_list)
df['zodiac'] = df[0].apply(lambda x: str(x).split("*")[0])
df['horo'] = df[0].apply(lambda x: str(x).split("*")[1:])
df['horo'] = df['horo'].apply(lambda x: ''.join(x))
df.head(10)

,0,zodiac,horo
0,Овен*Последний понедельник уходящего года запо...,Овен,Последний понедельник уходящего года запомнитс...
1,Овен*Во вторник звёзды рекомендуют вам… выключ...,Овен,Во вторник звёзды рекомендуют вам… выключить м...
2,Овен*Сегодня у представителей огненной стихии ...,Овен,Сегодня у представителей огненной стихии возмо...
3,Овен*В этот день представителям огненной стихи...,Овен,В этот день представителям огненной стихии мож...
4,Овен*Сегодня Овнам вдруг захочется позвонить с...,Овен,Сегодня Овнам вдруг захочется позвонить старым...
5,"Овен*Все вокруг носятся, стараются, готовятся ...",Овен,"Все вокруг носятся, стараются, готовятся к пра..."
6,"Овен*Вся страна лежит и лениво потягивается, п...",Овен,"Вся страна лежит и лениво потягивается, погляд..."
7,Овен*,Овен,
8,Телец*Сегодня Тельцу захочется сделать столько...,Телец,"Сегодня Тельцу захочется сделать столько, будт..."
9,"Телец*День новостей, известий и перемен. В как...",Телец,"День новостей, известий и перемен. В какой-то ..."


In [37]:
df = df[df['zodiac'] != 'Mistake']

In [41]:
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('\xa0', ''))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('ОВЕН', 'Овен'))
df['zodiac'] = df['zodiac'].apply(lambda x: 'Дева' if x == '' else x)
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace(' ', ''))

In [42]:
df['zodiac'].value_counts()

Лев         2137
Рыбы        1989
Козерог     1935
Телец       1756
Скорпион    1753
Близнецы    1746
Овен        1744
Весы        1732
Стрелец     1697
Рак         1682
Водолей     1554
Дева        1288
Name: zodiac, dtype: int64

In [44]:
from datetime import date
df['date'] = date.today()
df['source'] = 'https://sibmama.ru/'
df = df[['date','zodiac', 'horo', 'source']]
df.head(5)

,date,zodiac,horo,source
0,2023-08-10,Овен,Последний понедельник уходящего года запомнитс...,https://sibmama.ru/
1,2023-08-10,Овен,Во вторник звёзды рекомендуют вам… выключить м...,https://sibmama.ru/
2,2023-08-10,Овен,Сегодня у представителей огненной стихии возмо...,https://sibmama.ru/
3,2023-08-10,Овен,В этот день представителям огненной стихии мож...,https://sibmama.ru/
4,2023-08-10,Овен,Сегодня Овнам вдруг захочется позвонить старым...,https://sibmama.ru/


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21013 entries, 0 to 21411
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    21013 non-null  object
 1   zodiac  21013 non-null  object
 2   horo    21013 non-null  object
 3   source  21013 non-null  object
dtypes: object(4)
memory usage: 820.8+ KB


Информация обработана, приведена в форму для анализа. Добавим ее в общий файл, horo1.csv, куда собираются все данные

In [46]:
df1 = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
df1.head(5)

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/


In [48]:
new = pd.concat([df1, df])
new.to_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv', index=False)

##  2. Сайт https://sakhalife.ru/category/goroskop/

На данном сайте также есть архив гороскопов на 67 страницах, попробуем их спарсить с учетом знака зодиака

In [49]:
def get_url():
    for page in range(1,tot_pages+1):
        
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')
        url = f'https://sakhalife.ru/category/goroskop/page/{page}/'
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = {'cat-top-rows'})
        data2 = soup.find_all(class_ = {'cat-row3 cat-row'})
        data3 = soup.find_all(class_ = {'cat-row4 cat-row'})
        data_fin = data+data2+data3
        for item in data_fin:
            url = item.find_all('a', href = True)
            for item in url:
                if item.attrs['href'].find('https://sakhalife.ru/ezhednevnyj-goroskop') != -1:
                    url = item.attrs['href']
                    yield url
        
                
def get_item():
    for url in get_url():
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = {'post-content-inner'})
        for item in data:
            try:
                title = item.find_all('h2')
                title = title[1:-1]
                title_lst = []
                for x in title:
                    zodiac = x.text.split(' ')[0]
                    title_lst.append(zodiac)
            except:
                zodiac = ''
                title_lst.append(zodiac)
            try:
                test = item.find_all('p')
                test = test[2:-1]
                horo_lst = []
                for item in test:
                    horo_lst.append(item.text.replace('\xa0', ' '))
            except:
                horo_lst.append('')
                
            fin = list(zip(title_lst, horo_lst))
            #print(url)
            
            yield fin

In [50]:
tot_pages = 67
log_frequency = 10
save_frequency = 100
result = []
counter = 0

for item in get_item():
    result.extend(item)
    counter += 1
    if counter % save_frequency == 0:
        df = pd.DataFrame(result)
        df.to_csv('horo_temp.csv')
        print(f'{counter} pages saved in csv..')
df = pd.DataFrame(result)
df.to_csv('horo_fin.csv')
print(f'\nTotal pages saved: {counter}') 

100 pages saved in csv..
Log: working over page 10...
200 pages saved in csv..
Log: working over page 20...
300 pages saved in csv..
Log: working over page 30...
400 pages saved in csv..
500 pages saved in csv..
Log: working over page 40...
600 pages saved in csv..
Log: working over page 50...
700 pages saved in csv..
800 pages saved in csv..
Log: working over page 60...
900 pages saved in csv..

Total pages saved: 933


In [53]:
df[0].value_counts()

Телец       620
Близнецы    620
Рак         620
Лев         620
Дева        620
Весы        620
Скорпион    620
Стрелец     620
Козерог     620
Водолей     620
Овен        616
Рыбы        605
Name: 0, dtype: int64

In [54]:
df['source'] = 'https://sakhalife.ru/category/goroskop'
df = df.rename(columns={0:'zodiac', 1:'horo'})
df['date'] = date.today()
df = df[['date','zodiac', 'horo', 'source']]
df.head(5)

,date,zodiac,horo,source
0,2023-08-10,Овен,Хотя сегодня вам не терпится поговорить с друг...,https://sakhalife.ru/category/goroskop
1,2023-08-10,Телец,"Сегодня хороший день, если вы можете избежать ...",https://sakhalife.ru/category/goroskop
2,2023-08-10,Близнецы,"Сегодня Луна находится в вашем знаке, что дает...",https://sakhalife.ru/category/goroskop
3,2023-08-10,Рак,"Это тот день, когда то жарко, то холодно. Обсу...",https://sakhalife.ru/category/goroskop
4,2023-08-10,Лев,"В принципе, отношения с друзьями и членами гру...",https://sakhalife.ru/category/goroskop


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7421 entries, 0 to 7420
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    7421 non-null   object
 1   zodiac  7421 non-null   object
 2   horo    7421 non-null   object
 3   source  7421 non-null   object
dtypes: object(4)
memory usage: 232.0+ KB


In [56]:
df1 = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
df1.head(5)

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/


In [57]:
new = pd.concat([df1, df])
new.to_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv', index=False)

## 3. Сайт http://vuslon.ru/news/rubric/list/goroskop-na-kazhdyy-den

Есть небольшой архив ежедневных гороскопов до октября 2022 года. Попробуем скачать и его

In [2]:
def get_url():
    for page in range(1, tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')
        url = f'http://vuslon.ru/news/rubric/list/goroskop-na-kazhdyy-den/page/{page}'
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'all-news__list all-news__list_image')
        for item in data:
            item_url = item.find('a').attrs['href']
            yield item_url
                
def get_horo():
    for url in get_url():
        zod_lst = []
        horo_lst = []
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find(class_ = 'news-main-container__paragraph')
        
        desc= data.find_all('p')
        for item in desc:
            try: 
                horo = item.text
                horo = horo.replace('\r', '  ')
                lst = horo.split('  ')
                for h in lst:
                    h = h.replace('\n', '')
                    h = h.replace('\xa0', ' ')
                    
                    if (h == 'Реклама') or (h == '') or (h == 'Источник https://sovainfo.ru/') or (h == 'Источник "Гадалкин дом"'):
                        pass
                    else:
                        if h.lower().strip() in zodiac_lst:
                            zodiac = h.strip()
                            zod_lst.append(zodiac)
                            
                        else:
                            horo = h.strip()
                            horo_lst.append(horo)
                        
            except:
                zodiac = ''
                horo = ''

        
        yield zod_lst, horo_lst

In [3]:
zodiac_lst = ['близнецы', 'весы', 'водолеи', 'водолей', 'дева', 'девы', 'козерог', 'козероги', 'лев', 'львы', 'овен',
             'овны', 'рак', 'раки', 'рыбы', 'скорпион', 'скорпионы', 'стрелец', 'стрельцы', 'телец', 'тельцы']

log_frequency = 5
tot_pages = 20

total_zod = []
total_horo = []
for zod, horo in get_horo():
    total_zod.append(zod)
    total_horo.append(horo)

print(len(total_zod))
print(len(total_horo))

Log: working over page 5...
Log: working over page 10...
Log: working over page 15...
Log: working over page 20...
300
300


In [4]:
df = pd.DataFrame(columns = ['zodiac', 'horo'])
for i in range(len(total_zod)):
    temp = pd.DataFrame(list(zip(total_zod[i], total_horo[i])), columns = ['zodiac', 'horo'])
    df = pd.concat([df, temp])
df

,zodiac,horo
0,ОВНЫ,"Наступили дни, когда пора затянуть ремни потуж..."
1,ТЕЛЬЦЫ,Сегодня ждите полезную информацию для вашего б...
2,БЛИЗНЕЦЫ,На сегодняшний день сложилось впечатление неув...
3,РАКИ,Погодные изменения скажутся на самочувствии. В...
4,ЛЬВЫ,В этот день захочется сменить место работы. Пр...
...,...,...
7,СКОРПИОНЫ,Скорпионы могут оказаться перед каким-то непро...
8,СТРЕЛЬЦЫ,11 октября Стрельцы могут что-то забыть или пе...
9,КОЗЕРОГИ,11 октября Козерогам может понадобиться поддер...
10,ВОДОЛЕИ,11 октября у Водолеев может появиться неожидан...


In [8]:
df = df.drop_duplicates(subset='horo', keep = 'first', inplace = False)
df = df[df['horo'] != '']

In [14]:
df['zodiac'] = df['zodiac'].apply(lambda x: x.lower())
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('водолеи', 'Водолей'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('девы', 'Дева'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('козероги', 'Козерог'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('львы', 'Лев'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('раки', 'Рак'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('скорпионы', 'Скорпион'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('стрельцы', 'Стрелец'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('тельцы', 'Телец'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.replace('овны', 'Овен'))
df['zodiac'] = df['zodiac'].apply(lambda x: x.capitalize())

In [15]:
df['zodiac'].value_counts()

Овен        298
Весы        298
Стрелец     298
Водолей     298
Лев         297
Близнецы    296
Скорпион    296
Телец       295
Козерог     295
Рак         294
Рыбы        294
Дева        293
Name: zodiac, dtype: int64

In [16]:
df['source'] = 'http://vuslon.ru/news/rubric/list/goroskop-na-kazhdyy-den'
df['date'] = date.today()
df = df[['date','zodiac', 'horo', 'source']]
df.head(5)

,date,zodiac,horo,source
0,2023-08-11,Овен,"Наступили дни, когда пора затянуть ремни потуж...",http://vuslon.ru/news/rubric/list/goroskop-na-...
1,2023-08-11,Телец,Сегодня ждите полезную информацию для вашего б...,http://vuslon.ru/news/rubric/list/goroskop-na-...
2,2023-08-11,Близнецы,На сегодняшний день сложилось впечатление неув...,http://vuslon.ru/news/rubric/list/goroskop-na-...
3,2023-08-11,Рак,Погодные изменения скажутся на самочувствии. В...,http://vuslon.ru/news/rubric/list/goroskop-na-...
4,2023-08-11,Лев,В этот день захочется сменить место работы. Пр...,http://vuslon.ru/news/rubric/list/goroskop-na-...


In [17]:
df1 = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
df1.head(5)

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/


In [18]:
new = pd.concat([df1, df])
new.to_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv', index=False)

In [19]:
test = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
test

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/
...,...,...,...,...
32679,2023-08-11,Скорпион,Скорпионы могут оказаться перед каким-то непро...,http://vuslon.ru/news/rubric/list/goroskop-na-...
32680,2023-08-11,Стрелец,11 октября Стрельцы могут что-то забыть или пе...,http://vuslon.ru/news/rubric/list/goroskop-na-...
32681,2023-08-11,Козерог,11 октября Козерогам может понадобиться поддер...,http://vuslon.ru/news/rubric/list/goroskop-na-...
32682,2023-08-11,Водолей,11 октября у Водолеев может появиться неожидан...,http://vuslon.ru/news/rubric/list/goroskop-na-...


In [2]:
import re

def get_url():
    for page in range(1200, tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')
        url = f'https://mygazeta.com/category/%D0%B3%D0%BE%D1%80%D0%BE%D1%81%D0%BA%D0%BE%D0%BF/page/{page}?ysclid=llgqbm765s403390480'
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all('a', rel="bookmark")
        for item in data:
            if bool(re.search(pattern = re.escape('Полный текст'), string = item.text)):
                item_url = item['href']
                yield item_url
                
def get_horo():
    for url_item in get_url():
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data= soup.find_all('h3')
        for item in data:
            if bool(re.search(pattern = re.escape('['), string = item.text)):
                zodiac = item.text
                zodiac = re.sub(re.compile('[^А-Яа-я ]'), '', zodiac).strip()
                zodiac = zodiac.split(' ')[-1]
                horo = item.findNext('p').text.strip()
                yield zodiac, horo
        

In [3]:
df_columns = ['zodiac', 'horo']
df = pd.DataFrame(columns = df_columns)
save_frequency = 2000
log_frequency = 300
tot_pages = 4582

In [4]:
def to_csv():
    counter = 0  

    for item in get_horo():
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('mygazeta.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('mygazeta.csv')

    print(f'\nTotal items saved: {counter}') 

to_csv()

Log: working over page 1200...
2000 items saved in csv..
4000 items saved in csv..
6000 items saved in csv..
8000 items saved in csv..
10000 items saved in csv..
12000 items saved in csv..
14000 items saved in csv..
16000 items saved in csv..
18000 items saved in csv..
20000 items saved in csv..
22000 items saved in csv..
24000 items saved in csv..
Log: working over page 1500...
26000 items saved in csv..
28000 items saved in csv..
30000 items saved in csv..
32000 items saved in csv..
34000 items saved in csv..
36000 items saved in csv..
38000 items saved in csv..
40000 items saved in csv..
42000 items saved in csv..
Log: working over page 1800...
44000 items saved in csv..
46000 items saved in csv..
48000 items saved in csv..
50000 items saved in csv..
52000 items saved in csv..
54000 items saved in csv..
56000 items saved in csv..
58000 items saved in csv..
60000 items saved in csv..
62000 items saved in csv..
64000 items saved in csv..
66000 items saved in csv..
Log: working over pa

In [3]:
df = pd.read_csv('mygazeta.csv', index_col=0)
new = df.drop_duplicates(subset='horo', keep = 'first', inplace = False)
new['source'] = 'https://mygazeta.com'
new['date'] = date.today()
new = new[['date','zodiac', 'horo', 'source']]
new.head(5)

,date,zodiac,horo,source
0,2023-08-20,Овен,"Плодотворный день, подходящий для многих дел. ...",https://mygazeta.com
1,2023-08-20,Овен,День завтра благоприятен для новых знакомств. ...,https://mygazeta.com
2,2023-08-20,Овен,"Вы будете более открыто, чем обычно, вести себ...",https://mygazeta.com
3,2023-08-20,Овен,Хотя ваше финансовое положение не вызывает опа...,https://mygazeta.com
4,2023-08-20,Овен,"Овнам необходимо экономить жизненную энергию, ...",https://mygazeta.com


In [4]:
df1 = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
df1.head(5)

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/


In [6]:
upd = pd.concat([df1, new])
upd.to_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv', index=False)

In [7]:
test = pd.read_csv('C:/Users/Kate/Desktop/Docker/sqlite/horo1.csv')
test

,date,zodiac,horo,source
0,2023-08-06,Овен,"Намерения ваши прекрасны, но время для их осущ...",https://74.ru/horoscope/daily/
1,2023-08-06,Телец,Ваше сострадание и понимание бесконечны. Вы по...,https://74.ru/horoscope/daily/
2,2023-08-06,Близнецы,"Небольшое неудобство, которое вам придется пре...",https://74.ru/horoscope/daily/
3,2023-08-06,Рак,Привычные глазу лица и предметы вряд ли смогут...,https://74.ru/horoscope/daily/
4,2023-08-06,Лев,Сегодня вы будете равным среди великих. У вас ...,https://74.ru/horoscope/daily/
...,...,...,...,...
255044,2023-08-20,Рыбы,"Ваше красноречие, образованность, широта взгля...",https://mygazeta.com
255045,2023-08-20,Рыбы,Рыбам сегодня будет отлично удаваться имитация...,https://mygazeta.com
255046,2023-08-20,Рыбы,Сейчас стоит заняться поисками нового занятия ...,https://mygazeta.com
255047,2023-08-20,Водолей,"Несмотря на то, что вы способы искусно и ловко...",https://mygazeta.com


In [8]:
test['source'].value_counts()

https://mygazeta.com                                         147574
https://github.com/dkagramanyan/astro                         73477
https://sibmama.ru/                                           21013
https://sakhalife.ru/category/goroskop                         7421
http://vuslon.ru/news/rubric/list/goroskop-na-kazhdyy-den      3552
https://www.newsler.ru/horoscope                               1248
https://retrofm.ru/goroskop                                     208
https://horoscopes.rambler.ru                                   208
https://74.ru/horoscope/daily/                                  204
https://astroscope.ru/horoskop/ejednevniy_goroskop/             144
Name: source, dtype: int64